# Load model pytorch only


In [ ]:
import io, requests, cv2, numpy as np

url = "../images/aoi_restricted_LT05_CU_015008_20050830_20190107_C01_V0_-300225_2018565.jpg"
img_stream = io.BytesIO(requests.get(url).content)

In [ ]:
img = cv2.imdecode(np.fromstring(img_stream.read(), np.uint8), 1)

cv2.imshow("img", img)
cv2.waitKey(0)

In [ ]:
# bizarre error, the order of imports matters here. If DefaultPredictor isn't imported after numpy, etc. then kenrel restarts
import os
import skimage.io as skio
from skimage import img_as_ubyte, exposure
from PIL import Image as pilimg
import warnings
import numpy as np
from detectron2.config import get_cfg
from detectron2.engine.defaults import DefaultPredictor
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
import detectron2.data.transforms as T
import torch

def load_and_edit_cfg_for_inference(cfg_path= "../app/most-recent-best-model-and-config-dec-2020/config.yaml",
                                   model_weights_path = "../app/most-recent-best-model-and-config-dec-2020/model_0002799.pth"):
    cfg = get_cfg()    # obtain detectron2's default config
    cfg.CONFIG_NAME = '' # add new configs for your own custom components
    cfg.DATASET_PATH = ''
    cfg.merge_from_file(cfg_path)   # load values from a file
    cfg.MODEL.WEIGHTS = model_weights_path
    cfg.INPUT.MIN_SIZE_TEST = 800
    cfg.INPUT.MAX_SIZE_TEST = 1333
    model_cfg = cfg.clone()
    return model_cfg

def load_model_for_inference(cfg):
    model = build_model(cfg)  # returns a torch.nn.Module
    model.eval()

    checkpointer = DetectionCheckpointer(model)
    checkpointer.load(cfg.MODEL.WEIGHTS); # ; suppresses long output
    return model


def run_inference_on_image(cfg):
    aug = T.ResizeShortestEdge(
            [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST
        )
    
def process_image_data(image_data = "../images/aoi_restricted_LT05_CU_015008_20050830_20190107_C01_V0_-300225_2018565.jpg"
    if type(image_data) is str:
        arr = skio.imread(image_data)
        arr = arr[:,:,::-1] # assumes image is in BGR order, puts it in RGB order since model expects RGB
    elif type(image_data) is 
    
    
def prepare_inputs():
    
def run_inference_on_batch(model, ):

with torch.no_grad():
    height, width = arr.shape[:2]
    image = aug.get_transform(arr).apply_image(arr)
    image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
    inputs = {"image": image, "height": height, "width": width}
    predictions = model([inputs]*30)

cpu_output = predictions[0]["instances"].to("cpu")

In [ ]:
from detectron2.utils.visualizer import ColorMode
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
import matplotlib.pyplot as plt
metadata = MetadataCatalog.get("test")
vis_p = Visualizer(arr[:,:,::-1], metadata, instance_mode=ColorMode.SEGMENTATION)

# move to cpu
# instances = result['instances']
vis_pred_im = vis_p.draw_instance_predictions(cpu_output).get_image()

def show_im(image, ax):
    # Show area outside image boundaries.
    ax.axis('off')
    ax.imshow(image)
    return ax

plt.rcParams['font.size'] = 20
plt.rcParams['axes.linewidth'] = 2
plt.style.use("seaborn")
fig,ax = plt.subplots(figsize=(10,10))
show_im(vis_pred_im,ax)

In [ ]:
import torch
from torch.utils.cpp_extension import CUDA_HOME
print(torch.cuda.is_available(), CUDA_HOME)

In [ ]:
def read_and_rescale_tif(img_tile, clamp_low=262, clamp_high=1775): # Landsat 5 ARD .25 and 97.75 percentile range for Nebraska
    img_array = skio.imread(img_tile)
    img_array = exposure.rescale_intensity(img_array, in_range=(clamp_low, clamp_high))  # Landsat 5 ARD .25 and 97.75 percentile range.
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        img_array = img_as_ubyte(img_array)
    return img_array

def save_rescaled_jpeg(old_tif_path, jpeg_dir, rescaled_arr):
    img_pil = pilimg.fromarray(rescaled_arr)
    fid = os.path.basename(old_tif_path).split(".tif")[0]
    jpeg_path = os.path.join(jpeg_dir, fid + ".jpg")
    # Export chip images
    with open(Path(jpeg_path), 'w') as dst:
        img_pil.save(dst, format='JPEG', subsampling=0, quality=100)

In [ ]:
import detectron2
detectron2.__version__

In [ ]:
import torch
torch.__version__

In [ ]:
import torchvision
torchvision.__version__

Example request

In [4]:
import requests
import json
import base64

base_url = 'http://localhost:8081/v1/pytorch_api/'
detect_endpoint = 'classify'

async_url = base_url + detect_endpoint
# https://stackoverflow.com/questions/3715493/encoding-an-image-file-with-base64
def read_tif_as_base64(tif_path):
    with open(tif_path, "rb") as image_file:
        return base64.b64encode(image_file.read())
tif_path = "../images/aoi_restricted_LT05_CU_015008_20050830_20190107_C01_V0_-300225_2018565.jpg"
payload = {'image_bytes': read_tif_as_base64(tif_path),
           'date': "2005-08-30"
            }

# payload = json.dumps(payload)  # serialize the json payload

r = requests.post(async_url, data=payload)
print(r.text)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8081): Max retries exceeded with url: /v1/pytorch_api/classify (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f65c3df4eb8>: Failed to establish a new connection: [Errno 111] Connection refused',))